In [ ]:
!pip install efficientnet
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math, re, os
import tensorflow as tf
print(tf.__version__)
from kaggle_datasets import KaggleDatasets # Only on Kaggle
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import Xception, ResNet50V2, NASNetLarge, InceptionResNetV2, DenseNet121
from efficientnet.keras import EfficientNetB7
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Input, Conv2D, multiply, LocallyConnected2D, Lambda, BatchNormalization
from tensorflow.keras.models import Model, load_model
from sklearn.model_selection import train_test_split

In [ ]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = tpu_strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

In [ ]:
BASE_DIR = "/kaggle/input/ranzcr-clip-catheter-line-classification/"
df_train = pd.read_csv(os.path.join(BASE_DIR, "train.csv"), index_col=0)

In [ ]:
TARGET_SIZE = 768
CHANNELS = 3
BATCH_SIZE = 16 * REPLICAS
STEPS_PER_EPOCH = len(df_train) // BATCH_SIZE
OUTPUT_SIZE = 11

In [ ]:
def to_float32_2(image, label):
    max_val = tf.reduce_max(label, axis=-1,keepdims=True)
    cond = tf.equal(label, max_val)
    label = tf.where(cond, tf.ones_like(label), tf.zeros_like(label))
    return tf.cast(image, tf.float32), tf.cast(label, tf.int32)

def to_float32(image, label):
    return tf.cast(image, tf.float32), label

def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [1024,1024, 3]) # explicit size needed for TPU
    return image


def read_labeled_tfrecord(example):
    # Create a dictionary describing the features.
    LABELED_TFREC_FORMAT = {
        "StudyInstanceUID"           : tf.io.FixedLenFeature([], tf.string),
        "image"                      : tf.io.FixedLenFeature([], tf.string),
        "ETT - Abnormal"             : tf.io.FixedLenFeature([], tf.int64), 
        "ETT - Borderline"           : tf.io.FixedLenFeature([], tf.int64), 
        "ETT - Normal"               : tf.io.FixedLenFeature([], tf.int64), 
        "NGT - Abnormal"             : tf.io.FixedLenFeature([], tf.int64), 
        "NGT - Borderline"           : tf.io.FixedLenFeature([], tf.int64), 
        "NGT - Incompletely Imaged"  : tf.io.FixedLenFeature([], tf.int64), 
        "NGT - Normal"               : tf.io.FixedLenFeature([], tf.int64), 
        "CVC - Abnormal"             : tf.io.FixedLenFeature([], tf.int64), 
        "CVC - Borderline"           : tf.io.FixedLenFeature([], tf.int64), 
        "CVC - Normal"               : tf.io.FixedLenFeature([], tf.int64), 
        "Swan Ganz Catheter Present" : tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image']) 
    image= tf.image.resize(image, [IMAGE_SIZE[0],IMAGE_SIZE[0]])
    uid= example["StudyInstanceUID"]
    cvca = example["CVC - Abnormal"]
    cvcb = example["CVC - Borderline"]
    cvcn = example["CVC - Normal"]
    etta = example["ETT - Abnormal"]
    ettb = example["ETT - Borderline"]
    ettn = example["ETT - Normal"]
    ngta = example["NGT - Abnormal"]
    ngtb = example["NGT - Borderline"]
    ngti = example["NGT - Incompletely Imaged"]
    ngtn = example["NGT - Normal"]
    sgcp = example["Swan Ganz Catheter Present"]

    label  = [etta, ettb, ettn, ngta, ngtb, ngti, ngtn,cvca, cvcb, cvcn , sgcp]
    label=[tf.cast(i,tf.float32) for i in label]
    return image,label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT  = {
    "StudyInstanceUID" : tf.io.FixedLenFeature([], tf.string),
    "image" : tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    image= tf.image.resize(image, [IMAGE_SIZE[0],IMAGE_SIZE[0]])
    image_name = example['StudyInstanceUID']
    return image, image_name # returns a dataset of image(s)

def read_labeled_tf_record(filenames, labeled=True, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    return dataset

def data_augment(image, label):
    image = tf.image.random_flip_left_right(image , seed=SEED)
    image = tf.image.random_flip_up_down(image, seed=SEED)
    image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness
    image = tf.image.random_saturation(image, 0, 2, seed=SEED)
    image = tf.image.adjust_saturation(image, 3)
    
    #image = tf.image.central_crop(image, central_fraction=0.5)
    return image, label   

def get_training_dataset(dataset):
    dataset = dataset.shuffle(2048)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def count_data_items(filenames):
    #the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

def load_dataset(filenames, labeled = True, ordered = False):
    ignore_order = tf.data.Options()
    
    if not ordered:
        ignore_order.experimental_deterministic = False
    
    dataset = (tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO).with_options(ignore_order).
               map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls = AUTO))
    
    return dataset

GCS_DS_PATH = KaggleDatasets().get_gcs_path("ranzcr-clip-catheter-line-classification") # you can list the bucket with "!gsutil ls $GCS_DS_PATH"
print( "GCS_DS_PATH :{} ".format(GCS_DS_PATH))

IMAGE_SIZE = [TARGET_SIZE, TARGET_SIZE]
SEED = 555    
FOLDS = 3
EPOCHS = 30
FIRST_FOLD_ONLY = True

training_filenames = []
training_filenames.append(GCS_DS_PATH + '/train_tfrecords/*.tfrec')
TRAINING_FILENAMES = tf.io.gfile.glob(training_filenames)
TEST_FILENAMES = tf.io.gfile.glob(GCS_DS_PATH + '/test_tfrecords/*.tfrec')
NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
print("STEPS_PER_EPOCH {}".format(STEPS_PER_EPOCH))
print('Dataset: {} training images,  {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_TEST_IMAGES))

In [ ]:
def get_model():
    input_shape = (TARGET_SIZE, TARGET_SIZE, CHANNELS)
    in_lay = Input(input_shape)
    #conv_base = InceptionResNetV2(include_top = False, weights = 'imagenet', input_shape = input_shape)
    conv_base = Xception(include_top = False, weights = 'imagenet', input_shape = input_shape)
    #conv_base = DenseNet121(include_top = False, weights = 'imagenet', input_shape = input_shape)
    #conv_base = EfficientNetB7(include_top = False, weights = 'imagenet', input_shape = input_shape)

    conv_base.trainable = False
    pt_features = conv_base(in_lay)
    bn_features = BatchNormalization()(pt_features)
    # Credit: kmader https://www.kaggle.com/kmader/attention-on-pretrained-vgg16-for-bone-age
    # here we do an attention mechanism to turn pixels in the GAP on an off
    attn_layer = Conv2D(64, kernel_size = (1,1), padding = 'same', activation = 'relu')(bn_features)
    attn_layer = Conv2D(16, kernel_size = (1,1), padding = 'same', activation = 'relu')(attn_layer)
    attn_layer = LocallyConnected2D(1, kernel_size = (1,1), padding = 'valid', activation = 'sigmoid', implementation=2)(attn_layer)
    # fan it out to all of the channels
    pt_depth = conv_base.get_output_shape_at(0)[-1]
    up_c2_w = np.ones((1, 1, 1, pt_depth))
    up_c2 = Conv2D(pt_depth, kernel_size = (1,1), padding = 'same', 
                activation = 'linear', use_bias = False, weights = [up_c2_w])
    up_c2.trainable = False
    attn_layer = up_c2(attn_layer)

    mask_features = multiply([attn_layer, bn_features])
    gap_features = GlobalAveragePooling2D()(mask_features)
    gap_mask = GlobalAveragePooling2D()(attn_layer)
    # to account for missing values from the attention model
    gap = Lambda(lambda x: x[0]/x[1], name = 'RescaleGAP')([gap_features, gap_mask])
    gap_dr = Dropout(0.5)(gap)
    dr_steps = Dropout(0.25)(Dense(1024, activation = 'elu')(gap_dr))
    out_layer = Dense(11, activation = 'sigmoid')(dr_steps)
    model = Model(inputs = [in_lay], outputs = [out_layer])
    model.compile(optimizer = Adam(lr = 0.005), loss = 'binary_crossentropy', metrics = ["AUC"])
    return model

In [ ]:
from tensorflow.python.keras.callbacks import ReduceLROnPlateau
from tensorflow.python.platform import tf_logging as logging

# https://stackoverflow.com/questions/52227286/reducelronplateau-fallback-to-the-previous-weights-with-the-minimum-acc-loss
class ReduceLRBacktrack(ReduceLROnPlateau):
    def __init__(self, best_path, *args, **kwargs):
        super(ReduceLRBacktrack, self).__init__(*args, **kwargs)
        self.best_path = best_path

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if current is None:
            logging.warning('Reduce LR on plateau conditioned on metric `%s` '
                            'which is not available. Available metrics are: %s',
                             self.monitor, ','.join(list(logs.keys())))
        if not self.monitor_op(current, self.best): # not new best
            if not self.in_cooldown(): # and we're not in cooldown
                if self.wait+1 >= self.patience: # going to reduce lr
                    # load best model so far
                    print("Backtracking to best model before reducing LR")
                    self.model.load_weights(self.best_path)

        super().on_epoch_end(epoch, logs) # actually reduce LR

In [ ]:
def get_validation_dataset_for_kfold(dataset):
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def create_callbacks(model_save_path,fold):
    model_checkpoint_path = "{}/cmodel-{}.h5".format(model_save_path , fold)
    model_save = ModelCheckpoint(model_checkpoint_path, 
                                save_best_only = True, 
                                save_weights_only = True,
                                monitor = 'val_loss', 
                                mode = 'min', verbose = 1)

    early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.0001, 
                              patience = 5, mode = 'min', verbose = 1,
                              restore_best_weights = True)
    
    reduce_lr = ReduceLRBacktrack(best_path = model_checkpoint_path,
                                  monitor = 'val_loss', factor = 0.3, 
                                  patience = 2, min_delta = 0.0001, 
                                  mode = 'min', verbose = 1)
    """
    reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3,
                                  patience = 2, min_delta = 0.0001, 
                                  mode = 'min', verbose = 1)
    """
    callbacks = [model_save, early_stop, reduce_lr]
    return callbacks

In [ ]:
from sklearn.model_selection import KFold
import tensorflow.keras.backend as K

folds=5
#def train_cross_validate(folds = 5):
histories = []
models = []
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
kfold = KFold(folds, shuffle = True, random_state = SEED)
for f, (trn_ind, val_ind) in enumerate(kfold.split(TRAINING_FILENAMES)):
    print(); print('#'*25)
    print('### FOLD',f+1, "with trn_ind", trn_ind, "and val_ind", val_ind)
    print('#'*25)
    training_df = pd.DataFrame({'TRAINING_FILENAMES': TRAINING_FILENAMES})
    train_dataset = load_dataset(list(training_df.loc[trn_ind]['TRAINING_FILENAMES']), labeled = True)
    val_dataset = load_dataset(list(training_df.loc[val_ind]['TRAINING_FILENAMES']), labeled = True, ordered = True)
    K.clear_session()
    with tpu_strategy.scope():
        model = get_model()
        #model.load_weights('./cmodel-0.h5')
    validation_dataset = get_validation_dataset_for_kfold(val_dataset)

    print("Getting training dataset")
    training_dataset = get_training_dataset(train_dataset)

    model.summary()
    
    history = model.fit(
        training_dataset,
        steps_per_epoch = STEPS_PER_EPOCH,
        epochs = EPOCHS // 3,
        callbacks = create_callbacks(".", f),
        validation_data = validation_dataset
    )
    
    with tpu_strategy.scope():
        set_trainable = True
        for layer in model.layers: # Sets all layers to trainable except the one AFTER locally_connected2d
            print("Layer", layer.name, "set to trainable", set_trainable)
            layer.trainable = set_trainable
            set_trainable = not layer.name.startswith("locally") or not set_trainable
        model.compile(optimizer = Adam(lr = 0.0001), loss = "binary_crossentropy", metrics = ["AUC"])

    model.summary()
    history_ft = model.fit(
        training_dataset,
        steps_per_epoch = STEPS_PER_EPOCH,
        epochs = EPOCHS,
        callbacks = create_callbacks(".", f),
        validation_data = validation_dataset
    )

    scores = model.evaluate(validation_dataset)
    #print(scores)
    print(f'Score for fold {f+1}: {model.metrics_names[0]} of {scores[0]}')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    model.save("model-fold{}.h5".format(f+1))
    models.append(model)
    histories.append(history)
    if FIRST_FOLD_ONLY: break
    #return histories, models,acc_per_fold, loss_per_fold

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
for h in [history, history_ft]:
    auc = h.history['auc']
    val_auc = h.history['val_auc']
    loss = h.history['loss']
    val_loss = h.history['val_loss']

    epochs = range(1, len(auc) + 1)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    sns.set_style("white")
    plt.suptitle('Train history', size = 15)

    ax1.plot(epochs, auc, "bo", label = "Training auc")
    ax1.plot(epochs, val_auc, "b", label = "Validation auc")
    ax1.set_title("Training and validation auc")
    ax1.legend()

    ax2.plot(epochs, loss, "bo", label = "Training loss", color = 'red')
    ax2.plot(epochs, val_loss, "b", label = "Validation loss", color = 'red')
    ax2.set_title("Training and validation loss")
    ax2.legend()

    plt.show()

In [ ]:
def train_and_predict(folds = 5):
    test_ds = get_test_dataset(ordered=True) #map(data_augment, num_parallel_calls=AUTO) # since we are splitting the dataset and iterating separately on images and ids, order matters.
    test_images_ds = test_ds.map(lambda image, idnum: image)
    print('Start training %i folds'%folds)
    histories, models,acc_per_fold,loss_per_fold  = train_cross_validate(folds = folds)
    # == Provide average scores ==
    print('------------------------------------------------------------------------')
    print('Score per fold')
    for i in range(0, len(acc_per_fold)):
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
    print('------------------------------------------------------------------------')
    print('Average scores for all folds:')
    print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
    print(f'> Loss: {np.mean(loss_per_fold)}')
    print('------------------------------------------------------------------------')
    
    print('Computing predictions...')
    # get the mean probability of the folds models
    if FIRST_FOLD_ONLY: probabilities = np.average([models[i].predict(test_images_ds) for i in range(1)], axis = 0)
    else: probabilities = np.average([models[i].predict(test_images_ds) for i in range(folds)], axis = 0)
    
    return histories, models, probabilities, test_ds

In [ ]:
# Uncomment the below and the "def train_cross_validate" and "return" lines to run with all folds
#%%time
#histories, models, probabilities, test_ds = train_and_predict(folds = FOLDS)